# Komitety klasyfikatorów

Komitety (zespoły) klasyfikatorów to koncepcja kolektywnego połączenia wielu prostych klasyfikatorów, trenowanych w sposób zapewniający uzyskanie komplementarnych właściwości - wzajemnego uzupełniania swoich słabości, tak by uzyskać wypadkową dużą poprawność działania.  Isnieje kilka ogólnych pomysłów na strategię pracy zespołu. Pierwszym, jest zwielakratnianie (ang. bagging), zakładający wiele niezależnie działających komponentów składowych, gdzie wynikowa decyzja jest wypracowywana w drodze 'głosowania' (np. większościowego lub ważonego). Przykładem tej strategii jest model lasów losowych (decision forests), stanowiących kolekcję drzew decyzyjnych, trenowanych na różnych, losowo pobieranych podzbiorach zbioru treningowego. 
Innym pomysłem jest tzw. wzmacnianie (boosting), gdzie stosowana jest kaskada takich samych klasyfikatorów, gdzie każdy kolejny element kaskady próbuje naprawić błędy popełniane przez poprzedników. Głównym przedstawicielem tej strategii jest wzmacnianie adaptacyjne (Adaptive Boosting - AdaBoost), w którym kolejne klasyfikatory liniowe koncentrują swoją uwagę na wcześniej niewłaściwie klasyfikowanych przykładach treningowych.

## Lasy losowe

Lasy losowe to zbiory drzew decyzyjnych, uczonych na wielu losowo wybieranych podzbiorach zbioru dostępnych przykładów. Dla założonej liczby drzew, każde z nich jest trenowane $n-$elementowym podzbiorem $N-$elementowego zbioru wszystkich przykładów, z użyciem jednego ze znanych algorytmów (np. CART albo ID), przy czym dodatkowym elementem zwiększającym losowość procesu uczenia jest założenie, że dla każdego rozpatrywanego węzła drzewa, wybierany jest losowo podzbiór cech o zadanej liczności.  



L. Breiman, “Random forests,” Machine learning, vol. 45, no. 1, pp. 5–32, 2001

In [2]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100)

## AdaBoost

Adaptacyjne wzmacnianie to użycie sekwencji jednakowych klasyfikatorów, z których każdy następny próbuje zapewnić poprawną klasyfikację próbek, które wcześniej nie zostały poprawnie sklasyfikowane. Klasyfikatory składowe mogą być 'słabe' (ang. weak learners), czyli mogą zapewniać poprawność działania nawet niewiele lepszą niż losowe podejmowanie decyzji, mimo to kolektywne wyniki ich działania okazują się bardzo dobre. Koncepcja AdaBoost nie narzuca żadnych restrykcyjnych ograniczeń co do rodzaju klasyfikatorów - mogą to być drzewa decyzyjne, klasyfikatory liniowe lub Bayesa. Niezależnie od wyboru rodzaju klasyfikatorów, algorytm AdaBoost to iteracyjne powtarzanie następujących czynności:

- klasyfikacja bieżącego zestawu danych
- ocena wyniku klasyfikacji i przypisanie klasyfikatorowi 'znaczenia' (proporcjonalnego do poprawności działania). Jeżeli całkowity błąd popełniany przez klasyfikator oznaczymy jako $\epsilon $, miarą znaczenia jest wyrażenie:
\begin{equation}
\alpha = \frac{1}{2} \ln \frac{1-\epsilon}{\epsilon}
\end{equation}
Jeżeli błąd wyrażany jest jako liczba z zakresu [0...1], gdzie 0 odpowiada idealnej klasyfikacji, 1 - samym błędom, znaczenie klasyfikatora opisane powyższą funkcją jest krzywą o wartościach asymptotycznie dążących do nieskończoności (dla 0) i minus nieskończoności (dla 1), o punkcie przegięcia występującym dla błędu równego 0.5, dla którego znaczenie staje się zerowe.
- zróżnicowanie znaczenia próbek poprawnie i niepoprawnie klasyfikowanych: próbki błędnie klasyfikowane stają się 'ważniejsze', próbki poprawnie klasyfikowane - mniej ważne. Oznacza to wprowadzenie dla każdej próbki dodatkowej oceny jej znaczenia, czyli wagi. Na początku procedury, wszystkie próbki są jednakowo ważne, więc jeśli jest ich $n$, waga każdej z nich to $1/n$. Zmiana wagi próbki $j$ po '$i$-tym etapie klasyfikacji, jest funkcją znaczenia $Z$ klasyfikatora i ma postać:
\begin{equation}
w^{i+1}_j = w^i_j \exp^\alpha
\end{equation}
dla próbek klasyfikowanych nieprawdiłowo, i:
\begin{equation}
w^{i+1}_j = w^i_j \exp^{-\alpha}
\end{equation}
dla próbek klasyfikowanych poprawnie.
- powtarzanie procedury dla danych o uaktualnionych wagach do momentu uzyskania zadowalającej skuteczności lub uzyskania zadanej z góry liczby klasyfikatorów.


Reguła decyzyjna klasyfikatora AdaBoost to ważona przez znaczenie klasyfikatorów suma wyników ich decyzji - jeżeli decyzję $k$-tego klasyfikatora, o znaczeniu $\alpha_k$ odnośnie próbki $\mathbf{x}$ oznaczymy przez $h_k(\mathbf{x})$
to wynik zespołu klasyfikatorów jest określany jako:
\begin{equation}
y = \sum_k \alpha_k h_k(\mathbf{x})
\end{equation}


### AdaBoost z użyciem drzew decyzyjnych

Typowym sposobem realizacji koncepcji AdaBoost jest budowa lasów losowych, w których kolejne drzewa specjalizują się w analizie próbek, stanowiących problem dla drzew wcześniejszych. Ponieważ istotą metody AdaBoost jest używanie prostych klasyfikatorów, jako elementu lasu losowego najczęściej używa się skrajnie uproszczonej struktury drzewa, zawierającej tylko jeden poziom (jeden węzeł) i nazywanej często 'kikutem' (ang. stump). 

#### Przykład

Zakładając, że decyzja $Y$ jest podejmowana na podstawie trzech przesłanek ($A, B, C$) o binarnych wartościach, i że dane są następujące przykłady:
\begin{equation}
\begin{array}{cccc}
A & B & C & Y \\
0 & 0 & 1 & 0 \\
0 & 1 & 0 & 1 \\
0 & 1 & 1 & 1 \\
1 & 0 & 0 & 0 \\
1 & 0 & 1 & 1
\end{array}
\end{equation}

Budowa drzewa decyzyjnego to określenie, która cecha ma podlegać sprawdzeniu w pierwszym (i jedynym, w przypadku 'kikuta') węźle. Wybór cechy $A$ oznacza uzyskanie dwóch podzbiorów (2+ 1-) i (1+ 1-), wybór cechy $B$ to podzbiory (2- 1+) i (2+ 0-), wybór cechy $C$ to podzbiory (1+ 1-) i (2+ 1-). Bez konieczności przeprowadzania szczegółowych obliczeń, widać, że najlepiej porządkuje próbki wynik podziału względem wartości cechy $B$ i to ona zostaje wybrana jako test węzła w pierwszym 'kikucie' decyzyjnym. Taki klasyfikator będzie popełniał 1 błąd na pięć (błąd klasyfikatora $\epsilon = 0.2$), więc znaczenie klasyfikatora to:
\begin{equation}
\alpha_1 = \frac{1}{2} \ln \frac{1-0.2}{0.2} \approx 0.35
\end{equation}
Początkowo jednakowe wagi próbek (0.2) zostają więc zmodyfikowane w sposób uwypuklający ostatni (piąty) błędny przykład, a redukujący znaczenie przykładów poprawnie klasyfikowanych, zgodnie z zasadą:
\begin{equation}
w^2_5 = w^1_5 \exp^\alpha_1 = 0.2 e^{0.35} = 0.28
\end{equation}
\begin{equation}
w^2_{1...4} = w^1_{1...4} e^{-\alpha_1} = 0.2 e^{-0.35} = 0.14
\end{equation}


Próbka błędnie klasyfikowana nabiera w rozważanym przypadku dwukrotnie większego znaczenia niż próbki pozostałe. Ta informacja może być wykorzystana do treningu drugiego klasyfikatora (kikuta) na różne sposoby. Pierwszym z nich może być wykorzystanie wagi próbki jako dodatkowego mnożnika używanego w ocenie jakości podziału, określanej za pomocą współczynnika Giniego lub entropii. Drugim, powszechnie używanym w praktyce, jest generacja nowego zbioru danych o tej samej liczności co oryginalny, który odzwierciedla proporcje ważności próbek: próbki do nowego zbioru są dodawane z prawdopodobieństwem proporcjonalnym do ich znaczenia. W odniesieniu do rozważanego przykładu, oznacza to, że próbka ostatnia znajdzie się w nim dwukrotnie (w związku z tym. jedna z poprawnie klasyfikowanych, nie zostanie do niego włączona). Dla tak zbudowanego, nowego zbioru danych ponawiana jest procedura budowy drzewa. Proces ten trwa aż do uzyskania poprawnej klasyfikacji danych lub do osiągnięcia założonej z góry liczby elementów lasu losowego.

In [3]:
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier(n_estimators=100)